In [ ]:
from teomim import teomim

In [3]:
P=teomim(modelpath='./twin_models/FULL_QNET.joblib',alpha={1.3:1},
                 gz=False,outfile='out100.csv',num_patients=50)

#P.load('./twin_data/output.csv')

ModuleNotFoundError: No module named 'teomim'

In [1]:
%%time
P.generate_dist()

NameError: name 'P' is not defined

In [ ]:
ef=P.evaluate()
print(ef)

In [ ]:
print(P.quality())

In [ ]:
ef.plot(kind='bar')